In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [2]:
# Dependencies and Setup
import time
from scipy.stats import linregress
import warnings
warnings.filterwarnings('ignore')

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
df = pd.read_csv('C:/Users/rbvan/Desktop/python-api-challenge/output_data/cities.csv')

In [5]:
# Store latitude and longitude in locations
locations = df[['Lat', 'Lng']]

# Fill NaN values and convert to float
rating = df['Humidity'].astype(float)

In [6]:
# Quick review of humidity metrics
rating.describe()

count    545.000000
mean      70.133945
std       20.603185
min       14.000000
25%       58.000000
50%       74.000000
75%       86.000000
max      100.000000
Name: Humidity, dtype: float64

In [7]:
# Plot innitial heat map with humidity
fig = gmaps.figure()
hm_layer = gmaps.heatmap_layer(locations,weights=rating,max_intensity=100,dissipating=False,point_radius=1.2)
fig.add_layer(hm_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Create Data Fram with ideal weather conditions (temp between 70-80, less than 10 mph wind speed, and 0 cloudiness)

hotel_df = df[(df['Max Temp'] > 70) & (df['Max Temp'] < 80) & (df['Wind Speed'] < 10) & (df['Cloudiness'] == 0)]
hotel_df['Hotel Name'] = ''

In [9]:
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# params dictionary to update each iteration
params = {
    "radius": 100000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
# change location each iteration while leaving original params in place
# Include except for potential errors, print when hotel data is collected
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = (name_address['results'][0]["name"])
        print('Hotel Found')
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Hotel Found
Hotel Found
Hotel Found
Missing field/result... skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found


In [11]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Adding new layer on map for markers with hotel/city/country info
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))